In [60]:
from torch.utils.data import DataLoader,Dataset # 数据加载器
from sklearn.datasets import fetch_olivetti_faces
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [61]:
olivetti_faces = fetch_olivetti_faces(data_home='./face_data',shuffle=True)
# 定义自定义数据集类
class OlivettiFacesDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        return image, label
dataset=OlivettiFacesDataset(olivetti_faces.data,olivetti_faces.target)
# 创建数据加载器
batch_size = 64
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [62]:
# 定义神经网络模型
class FaceClassifier(nn.Module):
    def __init__(self, use_bn=True, use_dropout=True):
        super(FaceClassifier, self).__init__()
        self.use_bn = use_bn
        self.use_dropout = use_dropout

        # 定义网络层
        self.fc1 = nn.Linear(4096, 1024)  # 输入层到隐藏层
        self.fc2 = nn.Linear(1024, 512)   # 隐藏层到隐藏层
        self.fc3 = nn.Linear(512, 40)     # 隐藏层到输出层
        self.relu = nn.ReLU()             # 激活函数

        # 根据参数决定是否启用批归一化
        if self.use_bn:
            self.bn1 = nn.BatchNorm1d(1024)  # 批归一化
            self.bn2 = nn.BatchNorm1d(512)   # 批归一化
        else:
            self.bn1 = nn.Identity()         # 不启用批归一化
            self.bn2 = nn.Identity()         # 不启用批归一化

        # 根据参数决定是否启用 Dropout
        if self.use_dropout:
            self.dropout = nn.Dropout(0.5)   # Dropout 正则化
        else:
            self.dropout = nn.Identity()     # 不启用 Dropout

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展平图像
        x = self.fc1(x)
        x = self.bn1(x)            # 批归一化
        x = self.relu(x)
        x = self.dropout(x)        # Dropout
        x = self.fc2(x)
        x = self.bn2(x)            # 批归一化
        x = self.relu(x)
        x = self.dropout(x)        # Dropout
        x = self.fc3(x)
        return x

In [63]:
# 损失函数:交叉熵损失函数
criterion = nn.CrossEntropyLoss()
# 定义学习率
lR=1e-3

In [64]:
# 定义训练函数
def train_face_module(use_bn,use_dropout,optimizerType,num_epochs=10):
    # 记录训练过程中的 lossx
    train_losses = []
    # 设置模型为训练模式
    model=FaceClassifier(use_bn=use_bn,use_dropout=use_dropout)
    if optimizerType==1:
        optimizer=optim.Adam(model.parameters(), lr=lR);
    elif optimizerType==2:
        optimizer=optim.AdamW(model.parameters(), lr=lR);
    else:
        optimizer=optim.SGD(model.parameters(), lr=lR);
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            # 前向传播
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 记录损失和准确率
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # 计算平均损失和准确率
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.0 * correct / total

        train_losses.append(epoch_loss)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
        
    return epoch_loss

In [65]:
# 训练模型 
print("Adam优化器模型训练,不使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=True, optimizerType=1)
print(train_loss_history)
print("Adam优化器模型训练,使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=False, optimizerType=1)
print(train_loss_history)
print("Adam优化器模型训练,使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=True, optimizerType=1)
print(train_loss_history)
print("Adam优化器模型训练,不使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=False, optimizerType=1)
print(train_loss_history)

print("AdamW优化器模型训练,不使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=True, optimizerType=2)
print(train_loss_history)
print("AdamW优化器模型训练,使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=False, optimizerType=2)
print(train_loss_history)
print("AdamW优化器模型训练,使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=True, optimizerType=2)
print(train_loss_history)
print("AdamW优化器模型训练,不使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=False, optimizerType=2)
print(train_loss_history)



print("SGD优化器模型训练,不使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=True, optimizerType=3)
print(train_loss_history)
print("SGD优化器模型训练,使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=False, optimizerType=3)
print(train_loss_history)
print("SGD优化器模型训练,使用归一化使用正则化:")
train_loss_history = train_face_module(use_bn=True,use_dropout=True, optimizerType=3)
print(train_loss_history)
print("SGD优化器模型训练,不使用归一化不使用正则化:")
train_loss_history = train_face_module(use_bn=False,use_dropout=False, optimizerType=3)
print(train_loss_history)



Adam优化器模型训练,不使用归一化使用正则化:
Epoch [1/10], Loss: 4.1158, Accuracy: 1.50%
Epoch [2/10], Loss: 3.8907, Accuracy: 4.00%
Epoch [3/10], Loss: 3.7572, Accuracy: 1.50%
Epoch [4/10], Loss: 3.7320, Accuracy: 2.00%
Epoch [5/10], Loss: 3.7058, Accuracy: 2.25%
Epoch [6/10], Loss: 3.7081, Accuracy: 3.00%
Epoch [7/10], Loss: 3.7157, Accuracy: 4.25%
Epoch [8/10], Loss: 3.6798, Accuracy: 4.00%
Epoch [9/10], Loss: 3.6784, Accuracy: 3.50%
Epoch [10/10], Loss: 3.6662, Accuracy: 3.50%
3.666201489312308
Adam优化器模型训练,使用归一化不使用正则化:
Epoch [1/10], Loss: 2.3194, Accuracy: 52.25%
Epoch [2/10], Loss: 0.8525, Accuracy: 94.25%
Epoch [3/10], Loss: 0.3974, Accuracy: 99.25%
Epoch [4/10], Loss: 0.1740, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0984, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0651, Accuracy: 99.75%
Epoch [7/10], Loss: 0.0446, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0338, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0250, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0413, Accuracy: 99.75%
0.04133347115878548
Adam优化器模型训练,使用归